In [77]:
import os
import numpy as np

%load_ext autoreload
%autoreload 2

from datetime import datetime, timezone
from assignsvp import assignsvp as asvp
from cli import cli

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
def write_miniSeaAndSun_metadata(ctd_path, ctd_list_path, filetype):
    with open(ctd_path, 'r', encoding='ANSI') as f:
        file_content = f.read().splitlines()
        
        if filetype == 'ctd48':
            first_data_line = file_content[31].split()
        elif filetype == 'ctd90':
            first_data_line = file_content[34].split()
            
        date = first_data_line[1]
        time = first_data_line[2]
        
        datetime_start = datetime.strptime(f'{date}_{time}', '%d.%m.%Y_%H:%M:%S.%f')
        datetime_start.replace(tzinfo=timezone.utc)

        datetime1 = datetime_start.strftime('%Y-%m-%dT%H:%M:%S')
        ctd_name = filetype
        filename = os.path.split(ctd_path)[-1].split('.')[0]

        if os.path.exists(ctd_list_path):
            with open(ctd_list_path, 'a') as f:
                f.write(f"{datetime1}\t{ctd_name}\t{filename}\n")
        else:
            with open(ctd_list_path, 'w') as f:
                f.write('Datatime\tCTDName\tFname\n')
                f.write(f"{datetime1}\t{ctd_name}\t{filename}\n")

In [74]:
miniSVP_49 = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90'
logproc_dir = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\Обработка журнала станций'

SVP_path_list = cli.search(miniSVP_49, '.TOB')

Searching *.TOB files in directory:\\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90
Scanned filesets are: 

0 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_12.TOB
1 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_14.TOB
2 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_15.TOB
3 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_16.TOB
4 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_18.TOB
5 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_20.TOB
6 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_21.TOB
7 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_23.TOB
8 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_24.TOB
9 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\H5100534_25.TOB
10 \\10.197.125.15\Public\ABP53\abp

In [75]:
SVP_picked = cli.pick_from_list(SVP_path_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 
Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
H5100534_12.TOB
H5100534_14.TOB
H5100534_15.TOB
H5100534_16.TOB
H5100534_18.TOB
H5100534_20.TOB
H5100534_21.TOB
H5100534_23.TOB
H5100534_24.TOB
H5100534_25.TOB
H5100534_26.TOB
H5100534_27.TOB
H5100534_28.TOB
H5100534_29.TOB
H5100534_37.TOB
H5100534_5.TOB
H5100534_6.TOB
H5112304_1.TOB
H5112304_11.TOB
H5112304_13.TOB
H5112304_2.TOB
H5112304_3.TOB
H5112304_4.TOB
H5112304_5.TOB
H5112304_6.TOB
H5112304_7.TOB
H5112304_8.TOB
H5112304_9.TOB
H5172221_1.TOB
H5172221_11.TOB
H5172221_12.TOB
H5172221_13.TOB
H5172221_14.TOB
H5172221_15.TOB
H5172221_18.TOB
H5172221_19.TOB
H5172221_20.TOB
H5172221_3.TOB
H5172221_4.TOB
H5172221_8.TOB
H5172221_9.TOB


In [76]:
for path in SVP_picked:
    write_miniSeaAndSun_metadata(path, os.path.join(logproc_dir, 'ctd90_meta.txt'), 'ctd90')

# Преобразование .csv CTD90 от Виктора Кречика в PDS2000 .svp

In [ ]:
def convert_ctd90csv_to_PDSsvp(idronautcsv_path, PDSsvp_path):
        # Step 1. Read Sound Speed Manager .csv file
    pdepth = []
    pSV = []
    ptemp = []
    psal = []

    with open(idronautcsv_path, 'r') as f:
        # field order: depth, sound speed, salinity, temperature
        line_start = 35  # Previous lines are a header
        filecontent = f.read().splitlines()[line_start:]
        for line in filecontent:
            line_content = line.split(',')
            depth = float(line_content[2])
            sv = float(line_content[8].split(';')[0])
            temp = float(line_content[3])
            sal = float(line_content[4])
            if depth <= 0:
                pass
            elif sal < 1:
                pass
            elif len(pdepth) > 1:
                if len(pdepth) == 1:
                    pass
                elif pdepth[-1] >= depth and pdepth[-2] >= depth:
                    pass
                else:
                    pdepth.append(depth)
                    pSV.append(sv)
                    ptemp.append(temp)
                    psal.append(sal)
            else:
                pdepth.append(depth)
                pSV.append(sv)
                ptemp.append(temp)
                psal.append(sal)

    # Step 2. Create a new PDS2000 .svp and update it by SSM .csv values
    pheader = '[POINTS]\n'
    pnum_end = len(pdepth)
    offset_header = '\n[OFFSET]\n'
    sv_offset_name = 'SVOffset'
    sv_offset = '0'
    sv_offset_line = '{:} = {:}\n\n'.format(sv_offset_name,
                                        sv_offset)

    # field order: depth, sound velocity, temperature, salinity
    with open(PDSsvp_path, 'w') as f:
        f.write(pheader)
        for pnum in range(pnum_end):
            # field order: depth, sound velocity, temperature, salinity
            point = 'Point({:d}) = {:f},{:f},{:f},{:f}\n'.format(pnum+1,
                                                                 pdepth[pnum]*-1, 
                                                                 pSV[pnum], 
                                                                 ptemp[pnum], 
                                                                 psal[pnum])
            f.write(point)
        f.write(offset_header)
        f.write(sv_offset_line)

############################

In [ ]:
logpath = 'F:\\ABP_049\\Станции, логбуки и треки\\ABP049_CTD90_Log.txt'
datapath = 'F:\\ABP_049\\Станции, логбуки и треки\\CTD_Krechik\\CTD90'

sn = 'CTD90'
ctd90_csv_paths = find_file_with_extension('.CSV', datapath)

for ctd90_csv_path in ctd90_csv_paths:
    PDS_svp_path = modify_output_file_name(sn, logpath, ctd90_csv_path, filetype='CTD90')
    convert_ctd90csv_to_PDSsvp(ctd90_csv_path, PDS_svp_path)